# ⚡ Part 6: Efficient Workflows with ParallelAgent
## Content Creation Studio Workshop

Welcome to Part 6 of the Content Creation Studio Playbook! We've built an incredible system of agents that can delegate, follow ordered plans, and even iterate to improve quality. Now we tackle a new dimension: **efficiency**.

Our workflows so far have been linear - one step after another. What if we need to perform multiple, independent tasks **at the same time**? Today we'll learn about **ParallelAgent** - a workflow agent that runs sub-agents concurrently for maximum efficiency!

---

## 📚 The Content Creation Studio Playbook Series
- Part 1: Building Your First AI Agent ✅
- Part 2: Giving Your Agent Custom Tools ✅
- Part 3: Building Agent Teams with Agent-as-a-Tool ✅
- Part 4: Multi-Step Workflows with SequentialAgent ✅
- Part 5: Building Iterative Workflows with LoopAgent ✅
- **Part 6: Efficient Workflows with ParallelAgent** (You are here) 🎯
- Part 7: The Capstone Project

---

## 🎓 New Concepts in This Part

In Part 6, we'll introduce these **new ADK concepts**:

1. **ParallelAgent** - Workflow agent that runs all sub-agents concurrently
2. **Fan-Out/Fan-In Pattern** - Distribute work to multiple agents, then combine results
3. **Intake Agent Pattern** - Parse natural language into structured session state
4. **Session State Management** - Using `tool_context.state` to store workflow data

Each concept will be clearly marked with 🆕 when first introduced!

---

## 💡 What is Parallel Execution?

### 🆕 NEW CONCEPT: ParallelAgent (Workflow Agent)

> **What is ParallelAgent?**  
> A ParallelAgent is a workflow agent that executes **all of its sub-agents concurrently** (at the same time). It's designed for tasks that are independent of each other and don't need to run in a specific order. The ParallelAgent waits for all sub-agents to complete, then makes their outputs available to the next workflow step.
>
> **Key Characteristics**:
> - Runs all sub-agents simultaneously
> - Each sub-agent works independently
> - Total execution time = longest single sub-agent (not sum of all!)
> - Collects all results via `output_key` from each sub-agent
> - Parameters: `sub_agents` (list of agents to run in parallel)
>
> **Efficiency Gains**:
> - **Sequential**: Agent1 (5s) + Agent2 (5s) + Agent3 (5s) = 15s total
> - **Parallel**: All 3 agents run together = 5s total (longest one)
>
> **When to Use**:
> - Multiple independent tasks (blog + social + email)
> - Information gathering from multiple sources
> - Content creation for different channels
> - Any scenario where tasks don't depend on each other
>
> 📖 **Reference**: [Workflow Agents](https://google.github.io/adk-docs/agents/)

---

### 🆕 NEW CONCEPT: Fan-Out/Fan-In Pattern

> **What is Fan-Out/Fan-In?**  
> Fan-Out/Fan-In is an architectural pattern for parallel processing. **Fan-Out** means distributing a single input to multiple workers that process it simultaneously. **Fan-In** means collecting all those parallel results and combining them into a single output.
>
> **Architecture**:
> ```
> Single Input (Content Brief)
>       ↓
> [Fan-Out] → Distribute to Multiple Agents
>       ↓
> Agent1 | Agent2 | Agent3 | Agent4  ← All run in parallel
>       ↓
> [Fan-In] → Synthesizer Combines Results
>       ↓
> Single Output (Complete Package)
> ```
>
> **Components**:
> 1. **Input Agent** - Prepares/parses the input (optional)
> 2. **ParallelAgent** - Fan-out: Distributes work to sub-agents
> 3. **Specialist Agents** - Process independently in parallel
> 4. **Synthesizer Agent** - Fan-in: Combines all results
>
> **Benefits**:
> - Massive speed improvements
> - Scalable (add more parallel agents easily)
> - Clear separation of concerns
> - Each specialist focuses on one task
>
> **Real-World Example**:
> Content brief → [Blog Writer | Social Creator | Email Writer | SEO Agent] → Package Synthesizer
>
> 📖 **Reference**: [Workflow Agents](https://google.github.io/adk-docs/agents/)

---

## 🏗️ Our Workflow at a Glance

The system we're building today follows the **Fan-Out/Fan-In** pattern:

```
                    ┌────────────────┐
                    │  Content Brief │
                    └────────┬───────┘
                             │
                    ┌────────▼────────┐
                    │  Intake Agent   │ (Parse Brief)
                    └────────┬────────┘
                             │
                    ┌────────▼────────┐
                    │ Parallel Agent  │ (Fan Out)
                    └─┬────────────┬──┘
                      │            │
        ┌─────────────┼────────────┼─────────────┐
        │             │            │             │
        ▼             ▼            ▼             ▼
    ┌──────┐     ┌──────┐     ┌──────┐     ┌──────┐
    │ Blog │     │Social│     │Email │     │ SEO  │
    │Writer│     │Media │     │Writer│     │ Meta │
    └───┬──┘     └───┬──┘     └───┬──┘     └───┬──┘
        │             │            │             │
        └─────────────┼────────────┼─────────────┘
                      │
                      ▼
              ┌───────────────┐
              │ Synthesizer   │ (Fan In)
              └───────┬───────┘
                      │
                      ▼
              ┌───────────────┐
              │Content Package│
              └───────────────┘
```

### The Power of Parallel Execution:

All four content creators run **simultaneously**! If each takes 10 seconds, the total time is 10 seconds (not 40). This is how we achieve maximum efficiency.

!pip install google-adk==1.19.0 -q

In [ ]:
!pip install google-adk==1.8.0 -q

## 🔑 Setup: Configure Your API Key

Before we can build agents, we need to authenticate with Google's Gemini API.

### How to Get Your API Key
1. Navigate to [Google AI Studio](https://aistudio.google.com)
2. Sign in with your Google Account
3. Click the "Get API key" button
4. Follow the prompts to create a project
5. Copy your API key

**Note:** The Gemini API has a free tier. This tutorial won't cost you money!

In [ ]:
import os
from getpass import getpass
from IPython.display import display, Markdown

# Prompt for API key securely
api_key = getpass('Enter your Google API Key: ')
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully!")

## 🔧 1. Create an "Intake" Agent to Parse Content Briefs

### 🆕 NEW CONCEPT: Intake Agent Pattern

> **What is the Intake Agent Pattern?**  
> The Intake Agent Pattern is a design approach where a specialized agent parses natural language input from users and extracts structured parameters that other agents need. Instead of requiring users to provide structured input (JSON, forms, etc.), the intake agent interprets conversational requests and populates session state with the necessary data.
>
> **How It Works**:
> 1. User provides natural language input (e.g., "Create content about AI for small businesses, friendly tone")
> 2. Intake agent analyzes the request
> 3. Intake agent extracts key parameters (topic, audience, tone, keywords)
> 4. Intake agent stores parameters in session state via custom tool
> 5. Subsequent agents read from session state using `{{variable}}` syntax
>
> **Benefits**:
> - **User-friendly**: No structured input required
> - **Flexible**: Handles various ways of expressing requirements
> - **Robust**: Can ask clarifying questions if needed
> - **Separation of concerns**: Parsing logic isolated from execution logic
>
> **Implementation**:
> ```python
> intake_agent = Agent(
>     instruction="Extract topic, audience, tone, keywords and call update_session_state",
>     tools=[update_session_state]
> )
> ```
>
> 📖 **Reference**: [Multi-Agent Systems](https://google.github.io/adk-docs/agents/)

---

### 🆕 NEW CONCEPT: Session State Management

> **What is Session State?**  
> Session state (`tool_context.state`) is a key-value store that persists data throughout a conversation session. Tools can read from and write to this state, allowing data to flow between different agents in a workflow without manual parameter passing.
>
> **How to Use It**:
> ```python
> def my_tool(tool_context: ToolContext, param: str):
>     # Write to state
>     tool_context.state['my_key'] = param
>     
>     # Read from state
>     value = tool_context.state.get('my_key')
> ```
>
> **Access in Agent Instructions**:
> ```python
> Agent(
>     instruction="Write about: {{my_key}}"  # Reads from state
> )
> ```
>
> **Key Characteristics**:
> - Persists across agent executions within the same session
> - Accessible via `tool_context.state` in tools
> - Accessible via `{{variable}}` syntax in instructions
> - Supports any JSON-serializable data
>
> **Use Cases**:
> - Storing user preferences (tone, audience, keywords)
> - Passing data between workflow stages
> - Maintaining context across conversations
> - Building stateful workflows
>
> **Difference from output_key**:
> - `output_key`: Stores agent's final response text
> - `tool_context.state`: Stores arbitrary structured data from tools
>
> 📖 **Reference**: [Sessions](https://google.github.io/adk-docs/sessions/)

---

In previous parts, we sometimes hardcoded variables. A more robust approach is to have an agent extract information from the user's natural language query. We'll create an `intake_agent` using the Intake Agent Pattern.

First, we define a custom tool that saves extracted information to session state:

In [ ]:
from google.adk.tools import ToolContext
from google.adk.agents import Agent

def update_session_state(
    tool_context: ToolContext,
    topic: str,
    target_audience: str,
    tone: str,
    keywords: str
) -> str:
    """
    Saves the extracted content brief parameters to the session state.

    Parameters:
    - topic: The main topic of the content
    - target_audience: Who the content is for
    - tone: Writing tone (professional, casual, friendly, etc.)
    - keywords: Comma-separated keywords for SEO
    """
    print(f"🔧 Tool executed: Updating session state...")
    print(f"   Topic: {topic}")
    print(f"   Audience: {target_audience}")
    print(f"   Tone: {tone}")

    tool_context.state['topic'] = topic
    tool_context.state['target_audience'] = target_audience
    tool_context.state['tone'] = tone
    tool_context.state['keywords'] = keywords

    return "Session state has been updated with content brief parameters."

# This agent's only job is to parse the user's content brief
intake_agent = Agent(
    name="intake_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are a content brief analyzer. From the user's request, identify:
    - The main topic
    - The target audience
    - The desired tone (professional, casual, friendly, etc.)
    - Key SEO keywords (comma-separated)

    Then, you MUST call the `update_session_state` tool with the extracted values.
    """,
    tools=[update_session_state]
)

print("🧞 Intake agent created!")

## 👥 2. Create the Specialist Content Creator Agents

Now we define our specialists that will run in parallel. Their instructions use variables like `{{topic}}`, `{{target_audience}}`, and `{{tone}}`, which will be populated by our intake_agent.

We'll create four content creator agents:
1. **Blog post writer**
2. **Social media content creator**
3. **Email newsletter writer**
4. **SEO metadata generator**

Notice that each has its own `output_key` to store its output.

In [ ]:
# Agent 1: Blog Post Writer
blog_post_writer_agent = Agent(
    name="blog_post_writer_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are a professional blog writer. Write a complete blog post about: {{topic}}

    Guidelines:
    - Target audience: {{target_audience}}
    - Tone: {{tone}}
    - Length: 600-800 words
    - Include an engaging introduction
    - Use 3-4 H2 headings for main sections
    - Include actionable tips or insights
    - End with a strong call-to-action

    Output only the blog post in markdown format.
    """,
    tools=[],
    output_key="blog_post"
)

# Agent 2: Social Media Content Creator
social_media_creator_agent = Agent(
    name="social_media_creator_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are a social media content specialist. Create social posts about: {{topic}}

    Target audience: {{target_audience}}
    Tone: {{tone}}

    Create THREE posts:

    **1. LinkedIn Post** (150-200 words)
    - Professional and insightful
    - Start with a hook question or statistic
    - Include 3-4 professional hashtags

    **2. Twitter/X Thread** (3-4 tweets, 280 chars each)
    - Each tweet should stand alone but connect to form a thread
    - Include relevant hashtags
    - End with a call-to-action

    **3. Instagram Caption** (100-150 words)
    - Engaging and visual
    - Use emojis strategically
    - Include 8-10 hashtags at the end

    Format clearly with headers for each platform.
    """,
    tools=[],
    output_key="social_media_content"
)

# Agent 3: Email Newsletter Writer
email_newsletter_writer_agent = Agent(
    name="email_newsletter_writer_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are an email marketing specialist. Write an email newsletter about: {{topic}}

    Target audience: {{target_audience}}
    Tone: {{tone}}

    Structure:
    - **Subject Line**: Compelling and click-worthy (50-60 characters)
    - **Preview Text**: Enticing preview (40-50 characters)
    - **Body** (300-400 words):
      * Personal greeting
      * Engaging introduction
      * 2-3 key points with brief explanations
      * Clear call-to-action button text
      * Friendly sign-off

    Format with clear sections and headings.
    """,
    tools=[],
    output_key="email_newsletter"
)

# Agent 4: SEO Metadata Generator
seo_metadata_agent = Agent(
    name="seo_metadata_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are an SEO specialist. Generate SEO metadata for content about: {{topic}}

    Target keywords: {{keywords}}

    Create:
    1. **Meta Title** (50-60 characters, include main keyword)
    2. **Meta Description** (150-160 characters, compelling and keyword-rich)
    3. **URL Slug** (short, keyword-focused, lowercase with hyphens)
    4. **Focus Keyword**
    5. **5 Related Keywords** (for content optimization)
    6. **3 Internal Link Suggestions** (relevant topic ideas to link to)

    Format as a structured list.
    """,
    tools=[],
    output_key="seo_metadata"
)

print("🧞 All parallel content creator agents created!")

## ⚡ 3. Build the Parallel and Synthesizer Workflows

Here we build the core "fan-out, fan-in" logic. The **ParallelAgent** runs the four content creator agents concurrently. The `content_package_synthesizer_agent` then takes their outputs and combines them into a final content package.

This is where the magic happens - all four agents will execute at the same time!

In [ ]:
from google.adk.agents import ParallelAgent

# Create the parallel workflow
parallel_content_creation = ParallelAgent(
    name="parallel_content_creation",
    sub_agents=[
        blog_post_writer_agent,
        social_media_creator_agent,
        email_newsletter_writer_agent,
        seo_metadata_agent
    ]
)

# Agent to synthesize all parallel outputs
content_package_synthesizer_agent = Agent(
    name="content_package_synthesizer_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are a content package coordinator. Combine all the created content into one
    comprehensive package.

    You have:
    - Blog post: {{blog_post}}
    - Social media content: {{social_media_content}}
    - Email newsletter: {{email_newsletter}}
    - SEO metadata: {{seo_metadata}}

    Create a well-organized content package with:
    1. **📝 Blog Post** section
    2. **📱 Social Media Content** section
    3. **📧 Email Newsletter** section
    4. **🔍 SEO Metadata** section

    Present everything clearly with proper formatting and section headers.
    Add a brief executive summary at the top highlighting what's included.
    """
)

print("✅ Parallel workflow and synthesizer created!")

## 🏗️ 4. Assemble the Full Workflow

Finally, we assemble our complete, multi-step workflow using a **SequentialAgent**:

1. `intake_agent`: First, it parses the user's content brief and saves parameters
2. `parallel_content_creation`: Second, it fans out to create all content in parallel
3. `content_package_synthesizer_agent`: Third, it fans in to combine the results

This combines the power of sequential and parallel execution!

In [ ]:
from google.adk.agents import SequentialAgent

full_parallel_workflow = SequentialAgent(
    name="full_parallel_workflow",
    sub_agents=[
        intake_agent,
        parallel_content_creation,
        content_package_synthesizer_agent
    ],
    description="A workflow that parses content briefs, creates multiple content types in parallel, and synthesizes the results."
)

print("✅ Complete parallel content creation workflow assembled!")

## 🚀 5. Running the Full System

Our execution code is now clean and simple. We pass the user's natural language content brief, and the intake_agent handles the parameter extraction automatically.

Watch as the system:
1. Parses your content brief
2. Creates 4 different content pieces **simultaneously**
3. Packages everything together beautifully

In [ ]:
from google.adk.sessions import InMemorySessionService, Session
from google.adk.runners import Runner
from google.genai.types import Content, Part

# Initialize Session Service
session_service = InMemorySessionService()
user_id = "adk_content_creator_001"

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    print("\n" + "="*60)
    print("✅ FINAL CONTENT PACKAGE:")
    print("="*60)
    display(Markdown(final_response))
    print("="*60 + "\n")

    return final_response

async def run_parallel_content_creation():
    """Run the complete parallel content creation workflow"""
    session = await session_service.create_session(
        app_name=full_parallel_workflow.name,
        user_id=user_id
    )

    query = """
    Create a complete content package for:
    - Topic: Using AI tools to boost small business productivity
    - Target Audience: Small business owners and solopreneurs
    - Tone: Friendly and approachable, but professional
    - Keywords: AI productivity, small business automation, AI tools for business
    """

    print(f"👤 User Content Brief:\n{query}\n")

    await run_agent_query(full_parallel_workflow, query, session, user_id)

# Run the workflow
await run_parallel_content_creation()

## 🎯 Expected Output Flow

When you run this workflow, you'll see the parallel creation process:

### Step 1 - Intake Agent:
```
🔧 Updating session state...
   Topic: Using AI tools to boost small business productivity
   Audience: Small business owners and solopreneurs
   Tone: Friendly and approachable, but professional
```

### Step 2 - Parallel Content Creation:
All four agents run simultaneously:
```
📝 Blog Post Writer: Creating 700-word blog post...
📱 Social Media Creator: Creating posts for LinkedIn, Twitter, Instagram...
📧 Email Writer: Drafting newsletter with subject line...
🔍 SEO Specialist: Generating meta title, description, keywords...
```

### Step 3 - Content Package Synthesizer:
```
Combining all content into comprehensive package...

✅ FINAL CONTENT PACKAGE:
============================================================

## 📋 Executive Summary
Complete content package created for "Using AI tools to boost small business
productivity" targeting small business owners. Includes blog post (750 words),
multi-platform social content, email newsletter, and full SEO metadata.

## 📝 Blog Post
[Complete 700-word blog post with headings and CTA]

## 📱 Social Media Content
### LinkedIn Post
[Professional post with hashtags]

### Twitter/X Thread
[3-4 tweet thread]

### Instagram Caption
[Engaging caption with emojis and hashtags]

## 📧 Email Newsletter
**Subject:** [Compelling subject line]
**Preview:** [Preview text]
**Body:** [Full newsletter copy]

## 🔍 SEO Metadata
- Meta Title: [60-char optimized title]
- Meta Description: [160-char description]
- URL Slug: using-ai-tools-boost-business-productivity
- Focus Keyword: AI productivity tools
- Related Keywords: [5 keywords]
- Internal Links: [3 suggestions]
============================================================
```

## 🎉 Recap: What We've Learned

This was a major efficiency breakthrough in our workshop series!

### Core Concepts Introduced:

1. **🆕 ParallelAgent (Workflow Agent)** - Runs all sub-agents concurrently  
   Key: Total time = longest agent (not sum of all!)  
   📖 [Workflow Agents](https://google.github.io/adk-docs/agents/)

2. **🆕 Fan-Out/Fan-In Pattern** - Architectural pattern for parallel processing  
   Fan-Out: Distribute work to multiple agents  
   Fan-In: Synthesizer combines all results  
   📖 [Workflow Agents](https://google.github.io/adk-docs/agents/)

3. **🆕 Intake Agent Pattern** - Parse natural language into structured data  
   Benefit: User-friendly interface without requiring structured input  
   📖 [Multi-Agent Systems](https://google.github.io/adk-docs/agents/)

4. **🆕 Session State Management** - Using `tool_context.state` to store data  
   Access: `tool_context.state['key']` in tools, `{{key}}` in instructions  
   📖 [Sessions](https://google.github.io/adk-docs/sessions/)

### Key Takeaways:

- **Efficiency**: Parallel execution can dramatically reduce total workflow time
- **Scalability**: Easy to add more parallel agents without changing structure
- **User Experience**: Intake agents make systems more accessible
- **Real-World Patterns**: Fan-out/fan-in mirrors how professional teams work
- **State Management**: Session state enables flexible data flow

### Architecture Pattern:
```
Natural Language Input
       ↓
Intake Agent (parse & store in session state)
       ↓
ParallelAgent [Fan-Out]
       ├─ Agent1 (output_key="result1") ─┐
       ├─ Agent2 (output_key="result2") ─┤
       ├─ Agent3 (output_key="result3") ─┤ → All run simultaneously
       └─ Agent4 (output_key="result4") ─┘
       ↓
Synthesizer Agent [Fan-In]
       ↓
Combined Output
```

### Performance Comparison:

**Sequential Execution** (Parts 1-5):
```
Blog (10s) → Social (10s) → Email (10s) → SEO (10s) = 40 seconds total
```

**Parallel Execution** (Part 6):
```
Blog (10s) ┐
Social (10s)├─ All run together = 10 seconds total
Email (10s) ┤
SEO (10s)   ┘
```

4x faster! This is the power of ParallelAgent.

---

## 🎓 What's Next?

Congratulations! You've mastered all the core workflow patterns in Google ADK:
- ✅ **Part 1**: Basic Agents with tools
- ✅ **Part 2**: Custom Function Tools
- ✅ **Part 3**: Agent Teams (AgentTool, Orchestrator Pattern)
- ✅ **Part 4**: Sequential Workflows (ordered execution)
- ✅ **Part 5**: Iterative Workflows (quality loops)
- ✅ **Part 6**: Parallel Workflows (concurrent execution)

In **Part 7: The Capstone Project**, we'll combine **ALL** of these patterns into one comprehensive, production-ready Content Creation Studio. We'll build a master system that:
- Takes natural language content briefs (Intake Pattern)
- Researches topics with agent teams (AgentTool)
- Creates drafts through sequential steps (SequentialAgent)
- Improves quality through iterative loops (LoopAgent)
- Produces multi-channel content in parallel (ParallelAgent)
- Manages everything through hierarchical orchestration

This will be the ultimate demonstration of what you can build with Google ADK - a real-world, production-ready system combining all the patterns you've learned!

### Preview of Part 7 Concepts:
- 🆕 Hierarchical Orchestration
- 🆕 Composable Agent Systems
- 🆕 Production-Ready Architecture
- 🆕 End-to-End Autonomous Workflows

**See you in Part 7 for the grand finale! 🚀**